In [98]:
import pandas as pd
from constants import *
import numpy as np
import csv
from copy import deepcopy

In [3]:
def get_fnames(disease, pop_type, years):
    s = ["{}/{}.csv".format(OUTPUT_FILE, "{}_{}_{}".format(disease, year, pop_type)) for year in years]
    return s

In [9]:
years  =  ['2016', '2017', '2018']
fnames = get_fnames('ASTHMA', 'ADULT', years)

In [20]:
def get_average(dfs, column):
    t = []
    for df in dfs:
        summation = df[column].sum()
        t.append(summation)
    return round(np.mean(t))

In [21]:
get_average(dfs, 'at_risk')

454963532

In [100]:
dfs = []
for index, year in enumerate(years):
    df =  pd.read_csv(fnames[index])
    year_col = "POPEST{}_CIV".format(year)
    df['pop'] = df[year_col]
    df = df[~df['STATE'].isin(EXCLUDE_STATES)]
    dfs.append(df)

In [101]:
df.columns

Index(['STATE', 'SEX', 'CHILD', 'POPEST2018_CIV', 'State Code', 'State Name',
       'EPA Region', 'Prevalence', 'Standard', 'high_CI', 'low_CI', 'num',
       'count', 'incidence_rate', 'prevalence_cases', 'prevalence_cases_high',
       'prevalence_cases_low', 'at_risk', 'at_risk_high', 'at_risk_low',
       'incidence_cases', 'incidence_cases_high', 'incidence_cases_low', 'AF',
       'trap_incidence_cases', 'trap_incidence_cases_high',
       'trap_incidence_cases_low', 'pop'],
      dtype='object')

In [102]:
def get_prevalence_rate(dfs):
    prevalence_cases = get_average(dfs, 'prevalence_cases')
    population = get_average(dfs, 'pop')
    f = "{:.2f}".format(prevalence_cases*100/(population))
    return f

def get_prevalence_cases(dfs):
    prevalence_cases = round(get_average(dfs, 'prevalence_cases'))
    prevalence_cases_low = round(get_average(dfs, 'prevalence_cases_low'))
    prevalence_cases_high = round(get_average(dfs, 'prevalence_cases_high'))
    f = "{} ({}-{})".format(prevalence_cases, prevalence_cases_high, prevalence_cases_low)
    return f

def get_incidence_rate(dfs):
    at_risk = get_average(dfs, 'at_risk')
    incidences = get_average(dfs, 'incidence_cases')
    f = "{:.2f}".format(incidences*1000/(at_risk+incidences))
    return f

def get_incidence_cases(dfs):
    incidences = round(get_average(dfs, 'incidence_cases'))
    incidences_low = round(get_average(dfs, 'incidence_cases_low'))
    incidences_high = round(get_average(dfs, 'incidence_cases_high'))
    f = "{} ({}-{})".format(incidences, incidences_high, incidences_low)
    return f

def trap_incidences(dfs):
    trap_incidences = round(get_average(dfs, 'trap_incidence_cases'))
    trap_incidences_low = round(get_average(dfs, 'trap_incidence_cases_low'))
    trap_incidences_high = round(get_average(dfs, 'trap_incidence_cases_high'))
    f = "{} ({}-{})".format(trap_incidences, trap_incidences_high, trap_incidences_low)
    return f

def get_AF(dfs):
    trap_incidences = round(get_average(dfs, 'trap_incidence_cases'))
    incidences = round(get_average(dfs, 'incidence_cases'))
    res = "{:.2f}".format(trap_incidences*100/incidences)
    return res

In [105]:
headers = ["Region", "Standard", "Prevalence rate", "Prevalence cases", "Incidence rate", "Incidence cases", 
           "Incidences due to TRAP", "AF"]


def filter_standard(dfs):
    dfs_carb, dfs_noncarb = [], []
    for df in dfs:
        df_carb = df[df['STATE'].isin(CARB)]
        df_noncarb = df[~df['STATE'].isin(CARB)]
        dfs_carb.append(df_carb)
        dfs_noncarb.append(df_noncarb)
    return dfs_carb, dfs_noncarb

def filter_region(t_dfs, epa_region):
    dfs = deepcopy(t_dfs)
    for i in range(len(dfs)):
        dfs[i] = dfs[i][dfs[i]['EPA Region'] == epa_region]
    return dfs
        
def get_row(t_dfs):
    if not t_dfs[0].empty:
        row = [get_prevalence_rate(t_dfs), get_prevalence_cases(t_dfs), get_incidence_rate(t_dfs), get_incidence_cases(t_dfs),
               trap_incidences(t_dfs), get_AF(t_dfs)]
        return row
    return ['']*6

dfs_carb, dfs_noncarb = filter_standard(dfs)

f = open("Figure1_TABLE.csv", "w", newline='')
writer = csv.writer(f)
writer.writerow(headers)
for epa_region in range(0, 11):
    if epa_region == 0:
        t_carb = deepcopy(dfs_carb)
        t_noncarb = deepcopy(dfs_noncarb)
    else:
        t_carb = filter_region(dfs_carb, epa_region)
        t_noncarb = filter_region(dfs_noncarb, epa_region)
    row1 = [epa_region, 'CARB'] + get_row(t_carb)
    row2 = [epa_region, 'NonCARB'] + get_row(t_noncarb)
    writer.writerow(row1)
    writer.writerow(row2)
f.close()

In [104]:
dfs

[    STATE  SEX  CHILD  POPEST2016_CIV  State Code      State Name  EPA Region  \
 0       0    0      0       248090448         NaN             NaN         NaN   
 1       1    0      0         3750914         1.0         Alabama         4.0   
 2       2    0      0          533066         2.0          Alaska        10.0   
 3       4    0      0         5286902         4.0         Arizona         9.0   
 4       5    0      0         2278805         5.0        Arkansas         6.0   
 5       6    0      0        29922519         6.0      California         9.0   
 6       8    0      0         4242652         8.0        Colorado         8.0   
 7       9    0      0         2818731         9.0     Connecticut         1.0   
 10     12    0      0        16383575        12.0         Florida         4.0   
 11     13    0      0         7733313        13.0         Georgia         4.0   
 12     15    0      0         1072812        15.0          Hawaii         9.0   
 13     16    0 

In [93]:
dfs[0][dfs[0]['EPA Region'] == 6]


Empty DataFrame
Columns: [STATE, SEX, CHILD, POPEST2016_CIV, State Code, State Name, EPA Region, Prevalence, Standard, high_CI, low_CI, num, count, incidence_rate, prevalence_cases, prevalence_cases_high, prevalence_cases_low, at_risk, at_risk_high, at_risk_low, incidence_cases, incidence_cases_high, incidence_cases_low, AF, trap_incidence_cases, trap_incidence_cases_high, trap_incidence_cases_low, pop]
Index: []

[0 rows x 28 columns]